In [1]:
%matplotlib inline
import pandas as pd
import swifter
pd.set_option('display.max_rows', 10000)
import seaborn as sns
sns.set(rc={'figure.figsize':(13,13)})

In [2]:
train_articles = pd.read_csv("/data/semeval-2020/task-11/processed/train_article.csv")

In [3]:
span_lables = pd.read_csv("/data/semeval-2020/task-11/processed/span_labels.csv")

In [4]:
span_lables.head()

,article_id,start,end,label
0,776616374,149,174,Exaggeration_Minimisation
1,776616374,751,786,Exaggeration_Minimisation
2,776616374,831,849,Name_Calling_Labeling
3,776616374,2641,2650,Flag-Waving
4,776616374,2771,2782,Loaded_Language


In [5]:
span_lables["span_start_end_propaganda"] = span_lables.swifter.apply(lambda x: (x.start,x.end,x.label),axis=1)

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
article_with_label = train_articles.merge(span_lables,on="article_id")

In [7]:
article_with_label.head(3)

,article_id,content,start,end,label,span_start_end_propaganda
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...,630,669,Exaggeration_Minimisation,"(630, 669, Exaggeration_Minimisation)"
1,787529309,The Last-Minute Character Assassination of Jud...,70,88,Loaded_Language,"(70, 88, Loaded_Language)"
2,787529309,The Last-Minute Character Assassination of Jud...,288,320,Loaded_Language,"(288, 320, Loaded_Language)"


In [8]:
unique_article_span = article_with_label.groupby("article_id",as_index=False).agg(list)[["content","span_start_end_propaganda","article_id"]]

In [9]:
unique_article_span["content"] = unique_article_span.swifter.apply(lambda x:x.content[0],axis=1)

In [10]:
unique_article_span.head(3)

,content,span_start_end_propaganda,article_id
0,Next plague outbreak in Madagascar could be 's...,"[(265, 323, Appeal_to_Authority), (1795, 1935,...",111111111
1,US bloggers banned from entering UK Two promi...,"[(191, 221, Slogans), (476, 556, Black-and-Whi...",111111112
2,Kate Steinle's death at the hands of a Mexican...,"[(1396, 1430, Loaded_Language), (3082, 3099, N...",111111113


In [11]:
start_span_tag = " <span> "
end_span_tag = " </span> "

single_offset = len(start_span_tag) + len(end_span_tag)
print(single_offset)
def tag_article(row):
    article = row.content
    span_start_end_propaganda = row.span_start_end_propaganda
    span_start_end_propaganda.sort(key=lambda x:x[0])
    
    for ix,start_end_p in enumerate(span_start_end_propaganda):
        span_start = start_end_p[0] + (ix * single_offset) 
        span_end = start_end_p[1] + (ix * single_offset)
        print(start_end_p[0],span_start)
        print(start_end_p[1],span_end)
        article = article[:span_start-1] + start_span_tag + article[span_start-1:span_end] + end_span_tag +  article[span_end :] 
    return article    

17


In [12]:
a = unique_article_span.head(1).swifter.apply(lambda x: tag_article(x),axis=1)

149 149
157 157
265 282
323 340
1069 1103
1091 1125
1334 1385
1462 1513
1577 1645
1616 1684
1795 1880
1935 2020
1856 1958
1910 2012
2023 2142
2086 2205
149 149
157 157
265 282
323 340
1069 1103
1091 1125
1334 1385
1462 1513
1577 1645
1616 1684
1795 1880
1935 2020
1856 1958
1910 2012
2023 2142
2086 2205
149 149
157 157
265 282
323 340
1069 1103
1091 1125
1334 1385
1462 1513
1577 1645
1616 1684
1795 1880
1935 2020
1856 1958
1910 2012
2023 2142
2086 2205


149 149
157 157
265 282
323 340
1069 1103
1091 1125
1334 1385
1462 1513
1577 1645
1616 1684
1795 1880
1935 2020
1856 1958
1910 2012
2023 2142
2086 2205



In [13]:
a[0]

'Next plague outbreak in Madagascar could be \'stronger\': WHO  Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic <span>  appeared </span>  to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger.   <span> "The next transmission could be more pronounced or stronger </span> ," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious."  An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August.  Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires.  The disease tends to make a comeback each hot rainy season, from September to April. On average, between 300 and 600 infections are recorded every year among a population approaching 25 m

In [14]:
print([1,3,5,6,6,2,3,4,6,6][:3])
print([1,3,5,6,6,2,3,4,6,6][3:8])

[1, 3, 5]
[6, 6, 2, 3, 4]


In [15]:
article_with_label = train_articles.merge(span_lables,on="article_id")

In [16]:
def get_span(row):
#     print(row)
    article = row.content
    start_span = row.start
    end_span = row.end
    return (article[start_span:end_span],row.label)

In [17]:
article_with_label["span"] = article_with_label.swifter.apply(get_span,axis=1)

In [18]:
article_with_label.head()

,article_id,content,start,end,label,span_start_end_propaganda,span
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...,630,669,Exaggeration_Minimisation,"(630, 669, Exaggeration_Minimisation)","(the worst president in American history, Exag..."
1,787529309,The Last-Minute Character Assassination of Jud...,70,88,Loaded_Language,"(70, 88, Loaded_Language)","(despicable tactic , Loaded_Language)"
2,787529309,The Last-Minute Character Assassination of Jud...,288,320,Loaded_Language,"(288, 320, Loaded_Language)","(have sunk to their lowest level , Loaded_Lang..."
3,787529309,The Last-Minute Character Assassination of Jud...,924,961,Loaded_Language,"(924, 961, Loaded_Language)","(categorically and unequivocally deny , Loaded..."
4,787529309,The Last-Minute Character Assassination of Jud...,2405,2431,Loaded_Language,"(2405, 2431, Loaded_Language)","(cesspool of public gossip , Loaded_Language)"


## Create Dataset for sentences with propaganda yes or no

In [19]:
article_with_span = article_with_label.groupby("article_id",as_index=False).agg(set)[["article_id","content","span"]]

In [20]:
article_with_span["content"] = article_with_span["content"].swifter.apply(lambda x:list(x)[0],axis=1)
article_with_span["span"] = article_with_span["span"].swifter.apply(lambda x:list(x),axis=1)

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/swifter/swifter.py:172: UserWarning: Axis keyword not necessary because applying on a Series.
  warnings.warn("Axis keyword not necessary because applying on a Series.")


In [21]:
article_with_span.head()

,article_id,content,span
0,111111111,Next plague outbreak in Madagascar could be 's...,[(The next transmission could be more pronounc...
1,111111112,US bloggers banned from entering UK Two promi...,"[(Defeat Jihad"", Slogans), (so-called land of ..."
2,111111113,Kate Steinle's death at the hands of a Mexican...,"[(firestorm of outrage , Loaded_Language), (sa..."
3,111111114,U.S. judge frees Indonesian immigrant held by ...,[(they fear they could face discrimination or ...
4,111111115,Here are all the sexual misconduct accusations...,"[(aggressively stuck his tongue in my mouth"", ..."


In [40]:
import spacy; from spacy.lang.en import English; nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

def create_sentences(x):
    dataset = []
    doc = nlp(x.content)
    content = list(doc.sents)
    spans = set(x.span)
    print(content)
    print(spans)
    for i in content:
        print("===============================================")
        span_found = False
        label = 0
        for k in spans:
            print(i,"0000",k[0].strip())
            if k[0].strip() in str(i).strip():
                span_found = True
                label = k[1]
                break
        if span_found:
            dataset.append((i,label))
        else:
            dataset.append((i,label))
    return dataset        

In [ ]:
dataset = article_with_span.swifter.apply(lambda x: create_sentences(x),axis=1)

In [ ]:
len(article_with_span.iloc[1].content.split("."))

In [ ]:
article_with_span.iloc[1].span.split("$$$$$")

In [ ]:
len(dataset[1])

In [ ]:
count = 0
for sen in dataset[1]:
    if sen[1] == 1:
        count += 1

In [ ]:
count

In [ ]:
a = article_with_span.head(40).tail(1).apply(lambda x: create_sentences(x),axis=1)

In [31]:
article_with_span.head(40).tail(1)["content"][39]

'America\'s Immigration Voice.  The Kurds have no friends but the mountains, is an old lament . Last week, it must have been very much on Kurdish minds. As their U.S. allies watched, the Kurdish peshmerga fighters were run out of Kirkuk and all the territory they had captured fighting ISIS alongside the Americans. The Iraqi army that ran them out was trained and armed by the United States. The U.S. had warned the Kurds against holding the referendum on independence on Sept. 25, which carried with 92 percent. Iran and Turkey had warned against an independent Kurdistan that could be a magnet for Kurdish minorities in their own countries. But the Iraqi Kurds went ahead. Now they have lost Kirkuk and its oil, and their dream of independence is all but dead. More troubling for America is the new reality revealed by the rout of the peshmerga. Iraq, which George W. Bush and the neocons were going to fashion into a pro-Western democracy and American ally, appears to be as close to Iran as it i

In [29]:
article_with_span.head(40).tail(1)["span"][39]

[('what if the', 'Doubt'),
 ('apparently clueless to the presence of 800 U.S. troops in Niger,', 'Doubt'),
 ('secure, is that really a victory?', 'Doubt'),
 ('If we rely on the Kurds in Syria, we lose NATO ally Turkey,',
  'Black-and-White_Fallacy'),
 ('If he crosses, he joins Bush II in the history books.',
  'Bandwagon_Reductio_ad_hitlerum'),
 ("If we rely on the Kurds in Syria, we lose NATO ally Turkey, which regards Syria's Kurds as collaborators of the PKK in Turkey, which even the U.S. designates a terrorist organization. ",
  'Appeal_to_fear-prejudice'),
 ('interventionist Post', 'Name_Calling_Labeling'),
 ('Or do we leave our forces in Syria and Iraq and defy any demand from Assad to vacate his country? ',
  'Appeal_to_fear-prejudice'),
 ('Would the American people support such a war with U.S. troops? ',
  'Flag-Waving'),
 ('their dream of independence is all but dead.', 'Exaggeration_Minimisation'),
 ('terrorist"', 'Name_Calling_Labeling'),
 ('What if', 'Doubt'),
 ('But what i

In [42]:
for i in a[39]:
    if i[1] != 0:
        print(i)

(Now they have lost Kirkuk and its oil, and their dream of independence is all but dead., 'Exaggeration_Minimisation')
(But what if the Baghdad regime of Haider al-Abadi does not agree?, 'Doubt')
(What if the Quds Force does not go home to Iran and the Shiite militias that helped retake Kirkuk refuse to enlist in the Iraqi army?, 'Doubt')
(But if our victory in the war against ISIS leaves Iran in the catbird seat in Baghdad and Damascus, and its corridor from Tehran to Baghdad, Damascus and Beirut secure, is that really a victory?, 'Doubt')
(Or do we leave our forces in Syria and Iraq and defy any demand from Assad to vacate his country?, 'Appeal_to_fear-prejudice')
(Would the American people support such a war with U.S. troops?, 'Flag-Waving')
(Would Congress, apparently clueless to the presence of 800 U.S. troops in Niger, authorize a new U.S. war in Syria or Iraq?, 'Doubt')
(If we rely on the Kurds in Syria, we lose NATO ally Turkey, which regards Syria's Kurds as collaborators of t

In [32]:
sample_article = article_with_span.head(40).tail(1)["content"][39]

In [33]:
sample_article

'America\'s Immigration Voice.  The Kurds have no friends but the mountains, is an old lament . Last week, it must have been very much on Kurdish minds. As their U.S. allies watched, the Kurdish peshmerga fighters were run out of Kirkuk and all the territory they had captured fighting ISIS alongside the Americans. The Iraqi army that ran them out was trained and armed by the United States. The U.S. had warned the Kurds against holding the referendum on independence on Sept. 25, which carried with 92 percent. Iran and Turkey had warned against an independent Kurdistan that could be a magnet for Kurdish minorities in their own countries. But the Iraqi Kurds went ahead. Now they have lost Kirkuk and its oil, and their dream of independence is all but dead. More troubling for America is the new reality revealed by the rout of the peshmerga. Iraq, which George W. Bush and the neocons were going to fashion into a pro-Western democracy and American ally, appears to be as close to Iran as it i

In [100]:
def split_article_to_sentences(article):
    ci = re.finditer(r'\.\s+[A-Z]', article)
    indexes_to_break = [i.span()[0] for i in ci]

    sentences = []
    prev_index = 0
    for index_to_split in indexes_to_break:
        sentences.append(sample_article[prev_index:index_to_split])
        prev_index = index_to_split + 2

    return sentences
    
print(split_article_to_sentences(sample_article))

["America's Immigration Voice", ' The Kurds have no friends but the mountains, is an old lament ', 'Last week, it must have been very much on Kurdish minds', 'As their U.S. allies watched, the Kurdish peshmerga fighters were run out of Kirkuk and all the territory they had captured fighting ISIS alongside the Americans', 'The Iraqi army that ran them out was trained and armed by the United States', 'The U.S. had warned the Kurds against holding the referendum on independence on Sept. 25, which carried with 92 percent', 'Iran and Turkey had warned against an independent Kurdistan that could be a magnet for Kurdish minorities in their own countries', 'But the Iraqi Kurds went ahead', 'Now they have lost Kirkuk and its oil, and their dream of independence is all but dead', 'More troubling for America is the new reality revealed by the rout of the peshmerga', 'Iraq, which George W', 'Bush and the neocons were going to fashion into a pro-Western democracy and American ally, appears to be as

In [ ]:
import spacy; from spacy.lang.en import English; nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
doc = nlp(sample_article)
for sent in doc.sents:
    print("------")
    print(sent.text)